In [1]:

from pyspark.sql import SparkSession
from pykis import *
from itertools import chain 
import pandas as pd 
import numpy as np 
# from library import mariadb
# from library.trading_algorithms import EAVG, EAVG_vol
from datetime import datetime, timedelta 
from dateutil.relativedelta import relativedelta
import sys 
import os 
package_path = '/root/project/de-2024/jobs'
sys.path.append(package_path)
import cf

In [2]:
kis = PyKis(           
            appkey= cf.appkey
            ,appsecret= cf.appsecret
            ,virtual_account= True)

[06/23 21:03:29] INFO MARKET: up to date kospi
[06/23 21:03:29] INFO MARKET: up to date kosdaq
[06/23 21:03:29] INFO MARKET: up to date sector
[06/23 21:03:29] INFO RTC websocket connected


In [3]:
datetime(1985,1,1)

datetime.datetime(1985, 1, 1, 0, 0)

In [19]:
prices = kis.stock('005930').period_price(datetime(2022,1,1), datetime(2022,1,10))
stockdate = [p.stck_bsop_date.strftime("%Y%m%d") for p in prices.prices]

In [20]:
stockdate

['20220110', '20220107', '20220106', '20220105', '20220104', '20220103']

In [8]:
 stock = kis.stock('005930')

In [13]:
price = stock.price()

In [16]:
price

KisStockPrice()

In [5]:
stockdate

['19850105', '19850104']

In [26]:
import datetime
import json
import pandas as pd
import requests
import cf 
APP_KEY = cf.appkey
APP_SECRET = cf.appsecret
URL_BASE = "https://openapivts.koreainvestment.com:29443"  ## Pesudo


# Get authentication token.
# Refer https://apiportal.koreainvestment.com/apiservice/oauth2#L_fa778c98-f68d-451e-8fff-b1c6bfe5cd30
def auth():
  headers = {
      "content-type": "application/json"
  }

  body = {
      "grant_type":"client_credentials",
      "appkey":  APP_KEY,
      "appsecret": APP_SECRET
  }

  PATH = "oauth2/tokenP"
  URL = f"{URL_BASE}/{PATH}"

  res = requests.post(URL, headers = headers, data = json.dumps(body))

  global ACCESS_TOKEN
  ACCESS_TOKEN = res.json()["access_token"]


# Get price list for a ticker between from_date to to_date.
# Refer https://apiportal.koreainvestment.com/apiservice/apiservice-domestic-stock-quotations#L_a08c3421-e50f-4f24-b1fe-64c12f723c77
def get_prices_between(stock_no, from_date, to_date):
  PATH = "uapi/domestic-stock/v1/quotations/inquire-daily-itemchartprice"
  URL = f"{URL_BASE}/{PATH}"

  headers = {
      "Content-Type": "application/json",
      "authorization": f"Bearer {ACCESS_TOKEN}",
      "appKey": APP_KEY,
      "appSecret": APP_SECRET,
      "tr_id": "FHKST03010100"
  }

  params = {
      "fid_cond_mrkt_div_code": "J",
      "fid_input_iscd": stock_no,
      "fid_input_date_1":  ,
      "fid_input_date_2": to_date,
      "fid_period_div_code": "D",
      "fid_org_adj_prc": "0"
  }

  res = requests.get(URL, headers = headers, params = params)

  if res.status_code == 200 and res.json()["rt_cd"] == "0" :
    return(res.json())
  elif res.status_code == 200 and res.json()["msg_cd"] == "EGW00123" :
    auth()
    get_prices_between(stock_no, from_date, to_date)
  else:
    print("Error Code : " + str(res.status_code) + " | " + res.text)
    return None


def get_prices(ticker):
  now = datetime.datetime.now()

  from_dates = []
  for year in range(2000, now.year + 1):
    for month in range(1, now.month + 1 if year == now.year else 13, 4):
      from_dates.extend([(year, month)])

  prices = []
  empty_count = 0
  for year, month in reversed(from_dates):
    from_date = "%d%02d01" % (year, month)
    to_date = "%d%02d31" % (year, month + 3)
    res = get_prices_between(ticker, from_date, to_date)
    some_prices = list(filter(None, res["output2"]))
    some_prices_count = len(some_prices)
    print(f"{some_prices_count}", end = " ")
    if some_prices_count == 0:
      empty_count = empty_count + 1
    if empty_count > 1:
      break
    prices.extend(some_prices)

  return prices
  

In [29]:
auth()

In [32]:
try: ACCESS_TOKEN
except NameError: auth()


In [36]:
prices = get_prices_between('005930', '1985-01-01', '1985-01-31')



In [37]:
prices

{'output1': {'prdy_vrss': '-1600',
  'prdy_vrss_sign': '5',
  'prdy_ctrt': '-1.96',
  'stck_prdy_clpr': '81600',
  'acml_vol': '17907523',
  'acml_tr_pbmn': '1437355033344',
  'hts_kor_isnm': '삼성전자',
  'stck_prpr': '80000',
  'stck_shrn_iscd': '005930',
  'prdy_vol': '20288912',
  'stck_mxpr': '106000',
  'stck_llam': '57200',
  'stck_oprc': '80700',
  'stck_hgpr': '80800',
  'stck_lwpr': '80000',
  'stck_prdy_oprc': '81500',
  'stck_prdy_hgpr': '82200',
  'stck_prdy_lwpr': '81200',
  'askp': '80100',
  'bidp': '80000',
  'prdy_vrss_vol': '-2381389',
  'vol_tnrt': '0.30',
  'stck_fcam': '100',
  'lstn_stcn': '5969782550',
  'cpfn': '7780',
  'hts_avls': '4775826',
  'per': '37.54',
  'eps': '2131.00',
  'pbr': '1.54',
  'itewhol_loan_rmnd_ratem name': '0.14'},
 'output2': [{'stck_bsop_date': '20240621',
   'stck_clpr': '80000',
   'stck_oprc': '80700',
   'stck_hgpr': '80800',
   'stck_lwpr': '80000',
   'acml_vol': '17907524',
   'acml_tr_pbmn': '1437355033344',
   'flng_cls_code': '0

In [ ]:

tickers = (
    "069500",  # KODEX 200
    "114800",  # Samsung Kodex Inverse ETF
    "122630",  # Kodex Leverage
    "229200",  # KODEX KOSDAQ 150
    "233740",  # KODEX KOSDAQ150 Leverage
    "251340",  # KODEX KOSDAQ150 Inverse
    "252670",  # Samsung Kodex F Kospi 200 Inverse2x ETF
    "278530",  # Samsung Kodex 200 Total Return ETF
)

col_name_mapping = {
    "stck_bsop_date": "date",
    "stck_oprc": "open",
    "stck_hgpr": "high",
    "stck_lwpr": "low",
    "stck_clpr": "close",
    "acml_vol": "volume",
}


# Get access token if required.
try: ACCESS_TOKEN
except NameError: auth()

for ticker in tickers:
  print(f"For ticker {ticker}:", end = " ")

  print("Getting prices:", end = " ")
  prices = get_prices(ticker)

  print("Processing:", end = " ")
  items = filter(lambda x: len(x) > 0, prices)
  df = pd.DataFrame(items)
  df = df[col_name_mapping.keys()]
  df = df.astype(int)
  df.rename(columns = col_name_mapping, inplace = True)
  df.sort_values(by = ["date"], inplace = True)
  print("%d items" % len(df), end = " ")

  print("Writing to csv", end = " ")
  with open(f"drive/MyDrive/stock_data/{ticker}.csv", "w") as f:
    f.write(df.to_csv(index = False))
  print("- done")